In [1]:
import sqlite3
import numpy as np
import pandas as pd

In [3]:
class TicTacToeGame():
    def __init__(self, player1, player2):
        self.player1 = player1
        self.player2 = player2
        self.state = [[None, None, None],
                      [None, None, None],
                      [None, None, None]]
    
    def play(self, player1, player2, gameCount = -1, display = True):
        play = 'y'
        if gameCount <= -1:
            play = input("Play? (y/n)")
        while play == 'y' and gameCount != 0:
            winner = self.game(player1, player2)
            
            if winner == 1:
                player1.win()
                player2.lose()
            elif winner == 2:
                player1.lose()
                player2.win()
            else:
                player1.tie()
                player2.tie()
                
            if gameCount <= -1:
                play = input("Play? (y/n)")
            else:
                gameCount -= 1
            
    def game(self, player1, player2):
        while True:
            self.move(player1.move(self.state))
            finished = self.won()
            if finished: return finished
            self.move(player2.move(self.state))
            finished = self.won()
            if finished: return finished
            
    def move(self, move):
        pass
            
    def won(self):
        pass
    

In [38]:
class Player():
    def __init__(self):
        pass
    
    def move(self, state):
        raise NotImplementedError
    
class Human(Player):
    pass

class DataBot(Player):
    def __init__(self, ID):
        self.conn, self.cur = self.initiateDB(ID)
        
    def initiateDB(self, ID):
        conn = sqlite3.connect(str(ID) + '.db')
        cur = conn.cursor()
        try:
            c.execute("""CREATE TABLE states
                         ([state] string PRIMARY KEY, [wins] integer, [loss] integer, [tie] integer)""")
        except:
            pass
        conn.commit()
        return conn, cur
        
    def move(self, state):
        pass
    
class BoardHelper():
    def board_to_string(board):
        player1 = ''
        player2 = ''
        for i in range(len(board)):
            for j in range(len(board[i])):
                if board[i][j] == 1:
                    player1 += f'{i}{j}'
                elif board[i][j] == 2:
                    player2 += f'{i}{j}'
        return player1 + '/' + player2
    
    def string_to_board(string):
        board = [[None, None, None],
                 [None, None, None],
                 [None, None, None]]
        split = string.split('/')
        for i in range(len(split)):
            for pair in [split[i][n: n+2] for n in range(0, len(split[i]), 2)]:
                board[int(pair[0])][int(pair[1])] = i + 1
        return board
                

In [9]:
import sqlite3

conn = sqlite3.connect('TestDB.db')  # You can create a new database by changing the name within the quotes
c = conn.cursor() # The database will be saved in the location where your 'py' file is saved

# Create table - CLIENTS
try:
    c.execute('''CREATE TABLE B
                 ([state] STRING,[win] integer, [loss] integer, [tie] integer)''')
except:
    pass
                 
conn.commit()


In [14]:
c.execute("""REPLACE INTO B VALUES ('b',2,2,3)""")

In [15]:
c.execute("""SELECT * FROM B""")
c.fetchall()

[('d', 1, 2, 3), ('c', 2, 2, 3), ('a', 2, 2, 3), ('b', 2, 2, 3)]

In [8]:
conn.commit()